In [1]:
import requests 
import os
import re
from bs4 import BeautifulSoup 
import pandas as pd 
rss_feed_url = 'https://feeds.buzzsprout.com/1973304.rss'
page = requests.get(rss_feed_url)
soup = BeautifulSoup(page.content,'xml')
podcast_items = soup.find_all('item')

# Retrieve podcast information

In [2]:
base_url = 'https://podcasts.google.com/search/'

In [3]:
user_input = 'soccer' #user can input any word they want 

In [4]:
search_url = base_url + user_input 

resp = requests.get(search_url)
soup = BeautifulSoup(resp.text, 'lxml') #utilizes google podcast api to search for podcast results 

In [5]:
search_url

'https://podcasts.google.com/search/soccer'

In [ ]:
div_list = soup.find('div', role = 'list')
results = soup.find_all('a', {'role': 'listitem'}) #find the podcasts items inside of the soup content 
podcast_urls = []
domain_google_podcast = 'https://podcasts.google.com/'
for result in results: 
    podcast_url_part = result.get('href')[2:] #get the links of each podcast item 
#     podcast_url1 = domain_google_podcast+podcast_url_part
#     print(str(podcast_url1))
    podcast_urls.append(domain_google_podcast+podcast_url_part)
print(podcast_urls)

## Get RSS feed

In [ ]:
google_podcast_url = 'https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkcy5tZWdhcGhvbmUuZm0vVEFNQzE5NzkwMDE4NTk?sa=X&ved=0CAMQ4aUDahcKEwiI9OD5xrr9AhUAAAAAHQAAAAAQDg'

In [50]:
url_getrssfeed = 'https://getrssfeed.com'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

In [54]:
#to get podcast homepage url 
r = requests.post(url_getrssfeed, data={"url":google_podcast_url}, headers=headers) 
soup_getrssafterpost = BeautifulSoup(r.text, 'lxml')
results = soup_getrssafterpost.find_all('script', type='text/javascript')

for result in results:
    if 'http' in result.text:
        podcast_homepage_url_raw = result.text
        print("Find!")
        break
podcast_homepage_url = re.findall(r'"(.*?)"', podcast_homepage_url_raw)[0]
print("podcast homepage url = "+podcast_homepage_url)

In [103]:
#get podcast homepage url rss feed 
resp = requests.get(podcast_homepage_url)
soup = BeautifulSoup(resp.text, 'lxml')
a_list = soup.find_all('a',{"target":"_blank"})
for a in a_list:
    if a.text == "\n RSS feed\n":
        rss_url = a["href"]
        break

In [104]:
rss_url

'https://feeds.megaphone.fm/tamc1979001859'

## Parse RSS feed

## Create transcript 

In [ ]:
os.mkdir('.\downloads')

In [ ]:
def simplify_title(title):
    file_name = re.sub(r'[%/&!@#\*\$\?\+\^\\.\\\\]', '', title)[:100]
    return file_name
#file_name = title.replace('/','-').replace('\\\\','-').replace('.',' ')[:100]

In [ ]:
    title = podcast.find('title').text
    mp3_url = podcast.find('enclosure')['url']   

In [ ]:
mp3_url

In [ ]:
import requests, re
from bs4 import BeautifulSoup

website_links = ["https://www.sueddeutsche.de/"]

rss_feeds = []

def check_for_real_rss(url):
    base_url = re.search('^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/\n]+)',url).group(0)
    print(base_url)
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    for e in soup.select('[type="application/rss+xml"],a[href*=".rss"],a[href$="feed"]'):
        if e.get('href').startswith('/'):
            rss = (base_url+e.get('href'))
        else:
            rss = (e.get('href'))
        if 'xml' in requests.get(rss).headers.get('content-type'):
            rss_feeds.append(rss)
podcast = []
for url in website_links:
    soup = BeautifulSoup(requests.get(url).text)
    for e in soup.select('a[href*="rss"],a[href*="/feed"],a:-soup-contains-own("RSS")'):
        if e.get('href').startswith('/'):
            check_for_real_rss(url.strip('/')+e.get('href'))
        else:
            check_for_real_rss(e.get('href'))
    print(rss_feeds)

In [ ]:
count = 0 
for podcast in podcast_items:
    if count == 10:
        break
        
    title = podcast.find('title').text
    mp3_url = podcast.find('enclosure')['url']
    print(title+": "+mp3_url)
    mp3_file = requests.get(mp3_url)
    title = simplify_title(title).replace('|','').replace(' ','_')
    with open(f"downloads//{title}.mp3", 'wb') as f:
        f.write(mp3_file.content)
        print(title,count)
    count+=1

In [ ]:
def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data
            

In [ ]:
file_names

In [ ]:
headers = {'authorization': '4bec9f0712d7478f8a04495dea6afd94'}

#urls = []
files = 'downloads/'
file_names = os.listdir(files)
count = 0
mapping_table = []
for file in file_names:
    filename = f'{files}/{file}'
    response = requests.post('https://api.assemblyai.com/v2/upload',
                            headers=headers,
                            data=read_file(filename))
    
    upload_response = response.json()
    
    print("Transcription #", count)
    endpoint = "https://api.assemblyai.com/v2/transcript"
    json = {
        "audio_url": upload_response['upload_url'],
        "audio_start_from": 300000,
        "audio_end_at": 600000,
    }
    headers = {
        "authorization": '4bec9f0712d7478f8a04495dea6afd94',
        "content-type": "application/json"
    }
    transcription_response = requests.post(endpoint, json=json, headers=headers)
    print(transcription_response)
    transcription_id = transcription_response.json()['id']
    count+=1
    mapping_table.append([filename,upload_response['upload_url'],transcription_id])
    if(count==3):
        break
print(mapping_table)

In [ ]:
transcription_table = pd.DataFrame(mapping_table, columns=['filename','upload_url','transcript_id'])
transcription_table

In [ ]:
transcription_table['filename'] = transcription_table['filename'].str.replace('downloads//','')

In [ ]:
for index,row in transcription_table.iterrows():
    filename = row['filename']
    transcription_id = row['transcript_id']
    transcript_url = f'https://api.assemblyai.com/v2/transcript/{transcription_id}'

    transcription_result = requests.get(transcript_url, headers=headers)
    print(transcription_result.json()['text'])
    raw_text = transcription_result.json()['text']
    with open(f'./transcripts/{filename}.txt', 'w') as f:
        f.write(raw_text)